<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/EngtoVie_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing Data

###1, Import libraries

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd



###2, Reading the data

In [10]:
!wget http://www.manythings.org/anki/vie-eng.zip -O viet-eng.zip
!unzip viet-eng.zip

--2021-06-25 07:01:10--  http://www.manythings.org/anki/vie-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298429 (291K) [application/zip]
Saving to: ‘viet-eng.zip’

viet-eng.zip        100%[===================>] 291.43K  --.-KB/s    in 0.1s    

2021-06-25 07:01:10 (1.94 MB/s) - ‘viet-eng.zip’ saved [298429/298429]

Archive:  viet-eng.zip
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              
replace vie.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: u
error:  invalid response [u]
replace vie.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vie.txt                 


In [11]:
lines = pd.read_table( 'vie.txt' , names=[ 'eng' , 'vie' ] )
lines.reset_index( level=0 , inplace=True)
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'vie' , 'vie' : 'c' } , inplace=True )
lines = lines.drop('c',1)

In [12]:
lines.head()

,eng,vie
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!


### 3) Preparing input data for the Encoder ( `encoder_input_data` )

In [13]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()  
tokenizer.fit_on_texts( eng_lines ) 
tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1

### 4) Preparing input data for the Decoder ( `decoder_input_data` )

In [14]:
!pip install underthesea

  Using cached https://files.pythonhosted.org/packages/a8/5f/03ab9091b88e7851aa92da33f8eea6f111423cc1194cf1636c63c1fff3d0/underthesea-1.3.1-py3-none-any.whl
     |████████████████████████████████| 1.3MB 33.2MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 753.2MB 25kB/s 
  Using cached https://files.pythonhosted.org/packages/79/47/58f16c46506139f17de4630dbcfb877ce41a6355a1bbf3c443edb9708429/python_crfsuite-0.9.7-cp37-cp37m-manylinux1_x86_64.whl
     |████████████████████████████████| 245kB 56.5MB/s 
     |████████████████████████████████| 2.9MB 56.7MB/s 
     |████████████████████████████████| 901kB 47.2MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=421dce2be81da4bdc5a18aaf23214ed3db30c0209827cc9e4ff93af8239ea789
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully bui

In [15]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [16]:
from underthesea import word_tokenize
tokenized_vie_lines = []
dict_words = np.array([])
dict_numbers = np.array([]).astype('int64')
vie_lines = []
tokenized_vie_lines = []
vie_word_dict = dict()
for line in lines.vie:
    line = word_tokenize(line,format='text')  
    full_line = '<START> ' + line + ' <END>'
    full_line = cleanSentences(full_line)
    words = full_line.split()
    for word in words:
       where_word_arr = np.where(dict_words==word)[0]
       if len(where_word_arr)==0: 
           dict_words=np.append(dict_words,word)
           dict_numbers=np.append(dict_numbers,1)
       else: 
         dict_numbers[where_word_arr[0]]+=1
    vie_lines.append(full_line)

agr_sorted_dict_numbers = np.argsort(dict_numbers)[::-1]
for idx,value in enumerate(agr_sorted_dict_numbers):
    vie_word_dict.update({dict_words[value]:idx})

# tokenizer = preprocessing.text.Tokenizer()
# tokenizer.fit_on_texts(vie_lines) 
# tokenized_vie_lines = tokenizer.texts_to_sequences(vie_lines) 

# length_list = list()
# for token_seq in tokenized_vie_lines:
#     length_list.append( len( token_seq ))

# max_output_length = np.array( length_list ).max()
# print( 'Vietnamese max length is {}'.format( max_output_length ))

# padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length, padding='post' )
# decoder_input_data = np.array( padded_vie_lines )
# print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [20]:
for i,vie_line in enumerate(vie_lines):
    words = vie_line.split()
    words_arr = np.zeros(len(words),dtype=int)
    for i,word in enumerate(words):
       words_arr[i] = vie_word_dict[word]
    tokenized_vie_lines.append(words_arr)

In [22]:
length_list = list()
for token_seq in tokenized_vie_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array(length_list).max()
padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length , padding='post' )
decoder_input_data = np.array( padded_vie_lines )
num_vie_tokens = len(dict_words )+1
print( 'Number of Vietnamese tokens = {}'.format( num_vie_tokens))

Number of Vietnamese tokens = 3542


### 5) Preparing target data for the Decoder ( decoder_target_data ) 

In [47]:
decoder_target_data = list()
for token_seq in tokenized_vie_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_vie_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_vie_lines = utils.to_categorical( padded_vie_lines , num_vie_tokens)
decoder_target_data = np.array( onehot_vie_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (7547, 36, 3542)


#Defining and Training the models

###1, Defining the Encoder - Decoder model

In [48]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_eng_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 128 , return_state=True  )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_vie_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 128 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_vie_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    950272      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    906752      input_6[0][0]                    
____________________________________________________________________________________________

###Training model

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=50 ) 
model.save( 'model.h5' ) 

#Load weights

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/model_engtovie.h5')

In [51]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 128 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 128 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( eng_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [56]:
enc_model , dec_model = make_inference_models()

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter eng sentence : ' ) ) )
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = vie_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in vie_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter eng sentence : Hello
 start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start
Enter eng sentence : Run away
 start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start
Enter eng sentence : What are you doing
 start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start start


KeyboardInterrupt: ignored